In [1]:
import logging
from typing import List
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql import DataFrame
from pyspark.sql import types
from pyspark.sql.types import FloatType
from pyspark.sql.functions import year, col
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pandas as pd


In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

In [3]:
pyspark.__version__

'3.5.4'

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('pyspark-run-with-gcp-bucket') \
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/06/03 13:31:29 WARN Utils: Your hostname, jordi resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/03 13:31:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/03 13:31:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Set GCS credentials if necessary
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", "../keys/arxiv-trends-key.json")


In [6]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [7]:
df = spark.read.parquet("gs://test-arxiv-bucket-111865037319658112231/arxiv_data/arxiv_hep-ex_papers_2012_2018.parquet")


In [8]:
df.show()

25/06/02 18:21:35 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------------------+--------------------+---------------+--------------------+--------------------+----------------+--------------------+
|               title|             summary|submission_date|                  id|              author|primary_category|          categories|
+--------------------+--------------------+---------------+--------------------+--------------------+----------------+--------------------+
|Search for Extra ...|A possible soluti...|     2010-01-29|http://arxiv.org/...|  [Leonardo Benucci]| physics.ins-det|[physics.ins-det,...|
|K* resonance effe...|Charged and neutr...|     2010-01-29|http://arxiv.org/...|[O. Leitner, J. -...|          hep-ph|    [hep-ph, hep-ex]|
|Sivers and Boer-M...|Results for the n...|     2010-01-29|http://arxiv.org/...|[B. Pasquini, F. ...|          hep-ph|[hep-ph, hep-ex, ...|
|Triggering collec...|Collective flavor...|     2010-01-29|http://arxiv.org/...|[Basudeb Dasgupta...|          hep-ph|[hep-ph, astro-ph...|
|Measurement of Bc..

### Schema handling
Let's make sure the columns have the correct types

In [9]:
df.schema

StructType([StructField('title', StringType(), True), StructField('summary', StringType(), True), StructField('submission_date', DateType(), True), StructField('id', StringType(), True), StructField('author', ArrayType(StringType(), True), True), StructField('primary_category', StringType(), True), StructField('categories', ArrayType(StringType(), True), True)])

Take the first columns to get the types from it, and use pandas to create the spark df from it

In [10]:
df.limit(1000).toPandas().to_csv("head.csv", index=False)

In [11]:
df_pandas = pd.read_csv('head.csv')

In [12]:
df_pandas.dtypes

title               object
summary             object
submission_date     object
id                  object
author              object
primary_category    object
categories          object
dtype: object

In [13]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('title', StringType(), True), StructField('summary', StringType(), True), StructField('submission_date', StringType(), True), StructField('id', StringType(), True), StructField('author', StringType(), True), StructField('primary_category', StringType(), True), StructField('categories', StringType(), True)])

In [14]:
schema = types.StructType([
    types.StructField('title', types.StringType(), True),
    types.StructField('summary', types.StringType(), True),
    types.StructField('submission_date', types.DateType(), True),
    types.StructField('id', types.StringType(), True),
    types.StructField('author', types.ArrayType(types.StringType()), True),
    types.StructField('primary_category', types.StringType(), True),
    types.StructField('categories',  types.ArrayType(types.StringType()), True)
])

In [15]:
!rm head.csv

IOStream.flush timed out


Set the correct data types

In [16]:
df = spark.read \
    .schema(schema) \
    .parquet("gs://test-arxiv-bucket-111865037319658112231/arxiv_data/arxiv_hep-ex_papers_2012_2018.parquet")

2025-06-02 18:21:59,913 - INFO - Error while sending or receiving.
Traceback (most recent call last):
  File "/home/sabateri/anaconda3/envs/arxiv-trends/lib/python3.11/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer
2025-06-02 18:21:59,916 - INFO - Closing down clientserver connection
2025-06-02 18:21:59,917 - INFO - Exception while sending command.
Traceback (most recent call last):
  File "/home/sabateri/anaconda3/envs/arxiv-trends/lib/python3.11/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sabateri/anaconda3/envs/arxiv-trends/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = con

In [17]:
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- submission_date: date (nullable = true)
 |-- id: string (nullable = true)
 |-- author: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- primary_category: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [18]:
df.show()

+--------------------+--------------------+---------------+--------------------+--------------------+----------------+--------------------+
|               title|             summary|submission_date|                  id|              author|primary_category|          categories|
+--------------------+--------------------+---------------+--------------------+--------------------+----------------+--------------------+
|Search for Extra ...|A possible soluti...|     2010-01-29|http://arxiv.org/...|  [Leonardo Benucci]| physics.ins-det|[physics.ins-det,...|
|K* resonance effe...|Charged and neutr...|     2010-01-29|http://arxiv.org/...|[O. Leitner, J. -...|          hep-ph|    [hep-ph, hep-ex]|
|Sivers and Boer-M...|Results for the n...|     2010-01-29|http://arxiv.org/...|[B. Pasquini, F. ...|          hep-ph|[hep-ph, hep-ex, ...|
|Triggering collec...|Collective flavor...|     2010-01-29|http://arxiv.org/...|[Basudeb Dasgupta...|          hep-ph|[hep-ph, astro-ph...|
|Measurement of Bc..

### Define pyspark UDF's to apply to our data

In [19]:
# Define UDF
def clean_text(text: str) -> List[str]:
    """
    Clean and preprocess text by tokenizing, removing stopwords, and lemmatizing.
    
    Args:
        text: Raw text string to clean
        
    Returns:
        List of cleaned and lemmatized words
    """
    if not isinstance(text, str):
        logger.warning(f"Expected string but got {type(text)}. Converting to string.")
        text = str(text)
        
    text = text.lower()  # Convert to lowercase
    
    # Tokenize text into words
    try:
        words = word_tokenize(text)
    except Exception as e:
        logger.error(f"Error tokenizing text: {e}")
        return []
    
    # Remove stopwords and lemmatize
    cleaned_words = [
        lemmatizer.lemmatize(word) for word in words 
        if word.isalpha() and word not in stop_words
    ]
    
    return cleaned_words

In [20]:
def process_column(df: DataFrame, column: str) -> DataFrame:    
    """
    Clean the specified column and create a new column with the processed text.
    
    Args:
        df: DataFrame containing arXiv papers
        column: The column to clean ('title' or 'summary')
        
    Returns:
        DataFrame with added cleaned text column
        
    Raises:
        ValueError: If the specified column is not in the DataFrame
    """
    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    
    cleaned_column = f'cleaned_{column}'
    
    # Define a UDF to apply your clean_text function
    clean_text_udf = udf(clean_text, ArrayType(StringType()))
    
    logger.info(f"Processing column: {column}")
    df = df.withColumn(cleaned_column, clean_text_udf(col(column)))
    return df

In [21]:
df = process_column(df, 'title')
df = process_column(df, 'summary')

2025-06-02 18:22:01,727 - INFO - Processing column: title


2025-06-02 18:22:01,839 - INFO - Processing column: summary


In [22]:
def calculate_sentiment(df: DataFrame, column_name: str) -> DataFrame:
    """
    Calculate sentiment polarity score for text in the specified column in a Spark DataFrame.

    Args:
        df: Spark DataFrame containing arXiv papers
        column_name: Column name to analyze sentiment from

    Returns:
        Spark DataFrame with added 'sentiment' column

    Raises:
        ValueError: If the specified column is not in the DataFrame
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame.")
    
    sentiment_column = f'sentiment_{column_name}'

    # Define the UDF
    def compute_sentiment(text):
        if text is None:
            return 0.0
        try:
            return TextBlob(str(text)).sentiment.polarity
        except Exception:
            return 0.0

    sentiment_udf = udf(compute_sentiment, FloatType())
    
    logger.info(f"Sentiment analysis completed for column: {column_name}")
    df = df.withColumn(sentiment_column, sentiment_udf(col(column_name)))
    return df


In [23]:
df = calculate_sentiment(df, 'title')
df = calculate_sentiment(df, 'summary')

2025-06-02 18:22:01,958 - INFO - Sentiment analysis completed for column: title


2025-06-02 18:22:02,006 - INFO - Sentiment analysis completed for column: summary


In [24]:
df.show()

+--------------------+--------------------+---------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+---------------+-----------------+
|               title|             summary|submission_date|                  id|              author|primary_category|          categories|       cleaned_title|     cleaned_summary|sentiment_title|sentiment_summary|
+--------------------+--------------------+---------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+---------------+-----------------+
|Search for Extra ...|A possible soluti...|     2010-01-29|http://arxiv.org/...|  [Leonardo Benucci]| physics.ins-det|[physics.ins-det,...|[search, extra, d...|[possible, soluti...|           -0.1|     0.0020562771|
|K* resonance effe...|Charged and neutr...|     2010-01-29|http://arxiv.org/...|[O. Leitner, J. -...|          hep-ph|    [hep-ph, hep-e

Save the pyspark df into google cloud

In [25]:
# save all the output in one parquet file
# Coalesce the DataFrame to a single partition
df = df.coalesce(1)
output_path = "gs://test-arxiv-bucket-111865037319658112231/arxiv_data/processed/arxiv_hep-ex_papers_2012_2018_cleaned.parquet"
df.write.mode("overwrite").parquet(output_path)

We can also partition it in years

In [26]:

# Write partitioned Parquet

# Add a year column
# df = df.withColumn("year", year(col("submission_date")))

# df.write \
#     .partitionBy("year") \
#     .mode("overwrite") \
#     .parquet("gs://test-arxiv-bucket-111865037319658112231/arxiv_data/processed/")

In [5]:
# Stop SparkSession
spark.stop()